In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from dsmain.croptalk.rma_document import RmaDocument

from dsmain.dataapi.lookups import CommodityLookup, StateLookup, CountyLookup, InsurancePlanLookup

In [2]:
S3_BASE_URL = 'https://croptalk-spoi.s3.amazonaws.com/'

In [3]:
commodity_lookup = CommodityLookup()
state_lookup = StateLookup()
county_lookup = CountyLookup()
insurance_plan_lookup = InsurancePlanLookup()

In [4]:
def preprocess_webtables(filename, category):

    column_mapping = {
        'DocumentTitle': 'title',
        'InitialCropYear': 'year',
        'CommodityTags': 'commodity',
        'InsurancePlanTags': 'plan',
        'URL': 'source_url'
        }

    plan_names_map = {
        'Margin Protection - Crop': 'Margin Protection', 
        'Dollar Plan': 'Dollar Amount of Insurance',
        'Whole-Farm Revenue Protection': 'Whole Farm Revenue Protection',
        'Tree-Based Dollar Plan': 'Tree Based Dollar Amount of Insurance',
        'Group Risk Plan': 'Group Risk Plan, Area Yield Plan',
        #  'Commodity Exchange Price Provisions': '',
        #  'Basic Provisions': '',
        #  'Margin Price Provisions': '',
        #  'Livestock Gross Margin Swine': '',
        #  'Livestock Gross Margin Dairy': '',
    }

    commodity_names_map = {
        # 'All Commodities': '',
        # 'Commodities': '' 
    }

    # Load the data table parsed from the RMA website
    df = pd.read_csv(filename, dtype=str)
    df[df.isna()] = ''

    # Rename columns and drop unused columns
    df = df.rename(columns=column_mapping)
    df = df.drop(columns=['ValidCropYears', 'RelatedDocuments', 'Footnotes'])
    
    # Add necessary columns
    df['filename'] = df['source_url'].apply(lambda x: os.path.basename(x).split('.ashx')[0] + '.pdf')
    df['s3_key'] = category + '/' + df['filename']
    df['s3_url'] = S3_BASE_URL + df['s3_key']
    # df['s3_url'] = generate_presigned_urls(df['key'], bucket_name='cropguard-spoi')
    df['date_published'] = 'date_published'
    df['doc_category'] = category
    df['file_ext'] = 'pdf'
    df['county'] = ''
    df['state'] = ''

    # Replace 'All Commodities' and 'Commodities' with ''
    df['commodity'] = df['commodity'].apply(lambda x: x.replace('All Commodities', ''))
    df['commodity'] = df['commodity'].apply(lambda x: x.replace('Commodities', ''))

    # For columns policy and commodity with multiple values, split them into multiple rows by ;
    # WARNING: the docs with multi-tags will be duplicated for each policy and commodity
    for col in ['plan', 'commodity']:
        df[col] = df[col].apply(lambda x: x.split(';'))
        df = df.explode(col).reset_index(drop=True)
        df[col] = df[col].str.strip()
    
    # Replace plan and commodity names with the ones in the lookup table
    df['plan'] = df['plan'].replace(plan_names_map)
    df['commodity'] = df['commodity'].replace(commodity_names_map)

    df = df.drop_duplicates().reset_index(drop=True)
    print(df.shape)

    return df

In [8]:
def preprocess_sp(filename):
    column_mapping = {'key': 's3_key', 'policy':'plan'}
    dd = pd.read_csv(filename, dtype=str)
    dd = dd.rename(columns=column_mapping)
    dd = dd.drop(columns=['Unnamed: 0'])
    dd['commodity'] = dd['commodity'].str.zfill(4)
    dd['plan'] = dd['plan'].str.zfill(2)
    dd['county'] = dd['county'].str.zfill(3)
    dd['state'] = dd['state'].str.zfill(2)
    dd['title'] = ''
    dd['file_ext'] = 'pdf'
    dd['filename'] = dd['s3_key'].apply(lambda x: x.split('/')[-1])
    dd['s3_url'] = S3_BASE_URL + dd['s3_key']

    dd['title'] = 'Special Provisions for insuring ' + dd['commodity'] + \
        ' under plan ' + dd['plan'] + ' in state ' \
        + dd['state'] + ', county ' +  dd['county']
    return dd

In [9]:
# Process the webtables
dfs = []
files = [
    'Knowledge_20231117/CP_20231206.csv',
    'Knowledge_20231117/BP_20231206.csv',
    'Knowledge_20231117/CIH_20231206.csv'
]
for fname in files:
    category = os.path.basename(fname).split('_')[0]
    df = preprocess_webtables(fname, category)
    dfs.append(df)


# Add Special Provisions
df_sp = preprocess_sp('Knowledge_20231117/SP_20231206.csv')
dfs.append(df_sp)

df = pd.concat(dfs).reset_index(drop=True)
print(df.shape)
df.head()

(373, 13)
(44, 13)
(1, 13)
(56635, 13)


,title,year,commodity,plan,source_url,filename,s3_key,s3_url,date_published,doc_category,file_ext,county,state
0,Weaned Calf Crop Provisions 24-0805,2024,Weaned Calf,Yield Protection,https://www.rma.usda.gov/-/media/RMA/Policies/...,Weaned-Calf-Crop-Provisions-24-0805.pdf,CP/Weaned-Calf-Crop-Provisions-24-0805.pdf,https://croptalk-spoi.s3.amazonaws.com/CP/Wean...,date_published,CP,pdf,,
1,Weaned Calf Crop Provisions 24-0805,2024,Weaned Calf,Revenue Protection,https://www.rma.usda.gov/-/media/RMA/Policies/...,Weaned-Calf-Crop-Provisions-24-0805.pdf,CP/Weaned-Calf-Crop-Provisions-24-0805.pdf,https://croptalk-spoi.s3.amazonaws.com/CP/Wean...,date_published,CP,pdf,,
2,Weaned Calf Crop Provisions 24-0805,2024,Weaned Calf,Revenue Protection with Harvest Price Exclusion,https://www.rma.usda.gov/-/media/RMA/Policies/...,Weaned-Calf-Crop-Provisions-24-0805.pdf,CP/Weaned-Calf-Crop-Provisions-24-0805.pdf,https://croptalk-spoi.s3.amazonaws.com/CP/Wean...,date_published,CP,pdf,,
3,Walnut Crop Provisions 23-029,2023,Walnuts,Actual Production History,https://www.rma.usda.gov/-/media/RMA/Policies/...,Walnut-Crop-Provisions-23-029.pdf,CP/Walnut-Crop-Provisions-23-029.pdf,https://croptalk-spoi.s3.amazonaws.com/CP/Waln...,date_published,CP,pdf,,
4,Triticale Crop Provisions 22-TC-0158,2022,Triticale,Actual Production History,https://www.rma.usda.gov/-/media/RMA/Policies/...,Triticale-Crop-Provisions-22-TC-0158.pdf,CP/Triticale-Crop-Provisions-22-TC-0158.pdf,https://croptalk-spoi.s3.amazonaws.com/CP/Trit...,date_published,CP,pdf,,


In [14]:
df['doc_category'].value_counts()

doc_category
SP     56217
CP       373
BP        44
CIH        1
Name: count, dtype: int64

In [10]:
all_plans_code = '00'
all_commodities_code = '0000'
all_counties_code = '000'
all_states_code = '00'

unprocessed_commodities = []
unprocessed_plans = []
unprocessed_counties = []
unprocessed_states = []

registry = []

for _, row in df.iterrows():
    meta = row.to_dict()

    if not meta.get('plan'):
        # No plan specified => set code for all plans
        meta['plan'] = all_plans_code
    else:
        try:
            meta['plan'] = insurance_plan_lookup.find(meta['plan']).code
        except KeyError:
            # Plan not found in the lookup table => Skip the doc for now
            unprocessed_plans.append(meta['plan'])
            continue

    if not meta.get('commodity'):
        meta['commodity'] = all_commodities_code
    else:
        try:
            meta['commodity'] = commodity_lookup.find(meta['commodity']).code
        except KeyError:
            unprocessed_commodities.append(meta['commodity'])
            continue

    if not meta.get('state'):
        meta['state'] = all_states_code
    else:
        try:
            meta['state'] = state_lookup.find(meta['state']).code
        except KeyError:
            unprocessed_states.append(meta['state'])
            continue

    if not meta.get('county'):
        meta['county'] = all_counties_code
    else:
        try:
            county = county_lookup.find_by_code(meta['county'], meta['state'])
            if county:
                meta['county'] = county.code 
            else:
                unprocessed_counties.append((meta['state'], meta['county']))
                continue
        except KeyError:        
            unprocessed_counties.append((meta['state'], meta['county']))
            continue

    meta.pop('s3_key')
    registry.append(meta)
    
    # doc_rma = RmaDocument(**meta)
    # registry.append(doc_rma.to_dict())


In [11]:
print('Unprocessed commodities: ', set(unprocessed_commodities))
print('Unprocessed plans: ', set(unprocessed_plans))
print('Unprocessed counties: ', set(unprocessed_counties))
print('Unprocessed states: ', set(unprocessed_states))

Unprocessed commodities:  {'Bananas', 'Raspberries', '0247', '0252', 'Florida Citrus', 'Hybrid Seed Corn', 'Pasture-Rangeland-Forage', 'Pomegranate', '0205', 'Hawaii Tropical Trees', 'Cotton Extra Long Staple', 'Dairy', 'Fresh Market Peppers', 'Citrus', 'California Citrus Tree', '0237', 'Cotton Seed', 'Weaned Calf', '0225', 'Florida Fruit Tree', 'Tobacco', '0212', 'Pecan Tree', '0238', '0224', 'Small Grains', '0226', 'Sunflower Seeds', 'Fig', 'Processing Sweet Corn', 'Mustard Seed', '0284', 'Machine Harvested Pickling Cucumbers', 'Texas Citrus', '0228', '0214', '0804', 'Florida Avocados', 'Texas Citrus Tree', '0250', 'Coffee Trees', '0245', '0215', '0216', '0249', '9936', 'Sprinkler Irrigated Rice', 'Banana Trees', '0213', 'Cultivated Clams', '0206', '0217', 'Hawaii Tropical Fruit', 'Alfalfa', 'Processing Tomatoes', '0073', '0246', 'Downed Rice', 'Livestock', '0184', 'Processing Pumpkins', 'Sweet Cherry', 'Macadamia Tree', 'California Avocados', '0211', 'Apple Trees', 'Tart Cherry', 'A

In [12]:
registry_df = pd.DataFrame(registry)
registry_df['plan_commodity_county_state'] = registry_df['plan'] + '_' + registry_df['commodity'] + '_' + registry_df['county'] + '_' + registry_df['state']
registry_df.to_csv('Knowledge_20231117/registry_20231206_2.csv', index=False)

registry_df.tail()

,title,year,commodity,plan,source_url,filename,s3_url,date_published,doc_category,file_ext,county,state,plan_commodity_county_state
50055,Special Provisions for insuring 0049 under pla...,2024,0049,90,https://pubfs-rma.fpac.usda.gov/pub/Special_Pr...,30_091_90_0049_20231231.pdf,https://croptalk-spoi.s3.amazonaws.com/SPOI/30...,2023-12-31 00:00:00,SP,pdf,091,30,90_0049_091_30
50056,Special Provisions for insuring 0049 under pla...,2024,0049,90,https://pubfs-rma.fpac.usda.gov/pub/Special_Pr...,30_085_90_0049_20231231.pdf,https://croptalk-spoi.s3.amazonaws.com/SPOI/30...,2023-12-31 00:00:00,SP,pdf,085,30,90_0049_085_30
50057,Special Provisions for insuring 0049 under pla...,2024,0049,90,https://pubfs-rma.fpac.usda.gov/pub/Special_Pr...,30_083_90_0049_20231231.pdf,https://croptalk-spoi.s3.amazonaws.com/SPOI/30...,2023-12-31 00:00:00,SP,pdf,083,30,90_0049_083_30
50058,Special Provisions for insuring 0049 under pla...,2024,0049,90,https://pubfs-rma.fpac.usda.gov/pub/Special_Pr...,38_009_90_0049_20231231.pdf,https://croptalk-spoi.s3.amazonaws.com/SPOI/38...,2023-12-31 00:00:00,SP,pdf,009,38,90_0049_009_38
50059,Special Provisions for insuring 0086 under pla...,2024,0086,90,https://pubfs-rma.fpac.usda.gov/pub/Special_Pr...,51_131_90_0086_20231231.pdf,https://croptalk-spoi.s3.amazonaws.com/SPOI/51...,2023-12-31 00:00:00,SP,pdf,131,51,90_0086_131_51


In [13]:
registry_df.iloc[-1].to_dict()

{'title': 'Special Provisions for insuring 0086 under plan 90 in state 51, county 131',
 'year': '2024',
 'commodity': '0086',
 'plan': '90',
 'source_url': 'https://pubfs-rma.fpac.usda.gov/pub/Special_Provisions/2024/2024_SPOI_1231.zip',
 'filename': '51_131_90_0086_20231231.pdf',
 's3_url': 'https://croptalk-spoi.s3.amazonaws.com/SPOI/51_131_90_0086_20231231.pdf',
 'date_published': '2023-12-31 00:00:00',
 'doc_category': 'SP',
 'file_ext': 'pdf',
 'county': '131',
 'state': '51',
 'plan_commodity_county_state': '90_0086_131_51'}

In [46]:
registry_df['s3_url'].values[:10]

array(['https://croptalk-spoi.s3.amazonaws.com/CP/Walnut-Crop-Provisions-23-029.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Triticale-Crop-Provisions-22-TC-0158.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Table-Grape-Crop-Provisions-23-0052.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Sweet-Potato-Crop-Provisions-23-0156.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Sugarcane-Crop-Provisions-21-0038.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Strawberry-Pilot-Crop-Provisions-18-0154.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Stonefruit-Crop-Provisions-23-0077.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Stonefruit-Crop-Provisions-23-0077.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Stonefruit-Crop-Provisions-23-0077.pdf',
       'https://croptalk-spoi.s3.amazonaws.com/CP/Stonefruit-Crop-Provisions-23-0077.pdf'],
      dtype=object)